# Experiment 001: Baseline Evaluation and Optimization

This notebook:
1. Evaluates the existing baseline submission score
2. Implements fix_direction rotation optimization
3. Implements squeeze and compaction operations
4. Validates no overlaps

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Libraries loaded')

Libraries loaded


In [2]:
# Tree geometry constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

# Base tree polygon vertices (centered at origin, no rotation)
BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

print(f'Base tree has {len(BASE_TREE_VERTICES)} vertices')

Base tree has 15 vertices


In [3]:
def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x,y) with rotation deg degrees."""
    # Rotate vertices
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    
    return Polygon(translated)

def get_bounding_box_side(polygons):
    """Get the side length of the bounding box for a list of polygons."""
    if not polygons:
        return 0.0
    union = unary_union(polygons)
    bounds = union.bounds  # (minx, miny, maxx, maxy)
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

def check_overlaps(polygons):
    """Check if any polygons overlap (not just touch)."""
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        candidates = tree_index.query(poly)
        for j in candidates:
            if i < j:  # Only check each pair once
                if poly.intersects(polygons[j]) and not poly.touches(polygons[j]):
                    return True, (i, j)
    return False, None

print('Helper functions defined')

Helper functions defined


In [4]:
# Load baseline submission
baseline_df = pd.read_csv('/home/code/exploration/baseline_submission.csv')
print(f'Baseline submission shape: {baseline_df.shape}')
print(baseline_df.head())

Baseline submission shape: (20100, 4)
      id                       x                        y  \
0  001_0  s40.752900903586450454  s-32.002948532171380691   
1  002_0   s0.202513410337269301   s-0.028957664041420434   
2  002_1  s-0.105680728905459279   s-0.551876178651849569   
3  003_0   s1.127378112162989332    s0.792211449857787242   
4  003_1   s1.234055695842160016    s1.275999500663759001   

                       deg  
0   s45.000000000000035527  
1  s203.629377730656727863  
2   s23.629377730656813128  
3  s112.222533627590607352  
4   s66.370622269343002131  


In [5]:
# Parse submission values (remove 's' prefix)
def parse_submission(df):
    """Parse submission dataframe, removing 's' prefix from values."""
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

baseline_parsed = parse_submission(baseline_df)
print(baseline_parsed.head())

      id          x          y         deg
0  001_0  40.752901 -32.002949   45.000000
1  002_0   0.202513  -0.028958  203.629378
2  002_1  -0.105681  -0.551876   23.629378
3  003_0   1.127378   0.792211  112.222534
4  003_1   1.234056   1.276000   66.370622


In [6]:
# Calculate score for each configuration
def calculate_score(df):
    """Calculate the total score for a submission."""
    scores = {}
    total_score = 0.0
    
    for n in range(1, 201):
        # Get trees for this configuration
        prefix = f'{n:03d}_'
        config_df = df[df['id'].str.startswith(prefix)]
        
        if len(config_df) != n:
            print(f'Warning: Config {n} has {len(config_df)} trees instead of {n}')
            continue
        
        # Create polygons
        polygons = []
        for _, row in config_df.iterrows():
            poly = create_tree_polygon(row['x'], row['y'], row['deg'])
            polygons.append(poly)
        
        # Get bounding box side
        side = get_bounding_box_side(polygons)
        scores[n] = side
        total_score += (side ** 2) / n
    
    return total_score, scores

print('Calculating baseline score...')
baseline_score, baseline_sides = calculate_score(baseline_parsed)
print(f'Baseline score: {baseline_score:.6f}')

# Also calculate sample submission score
sample_df = pd.read_csv('/home/data/sample_submission.csv')
sample_parsed = parse_submission(sample_df)
sample_score, sample_sides = calculate_score(sample_parsed)
print(f'Sample submission score: {sample_score:.6f}')

Calculating baseline score...


Baseline score: 70.625918


Sample submission score: 173.652299


In [7]:
# Check which configurations have overlaps in baseline
print('Checking for overlaps in baseline...')
overlap_configs = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = baseline_parsed[baseline_parsed['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlap, pair = check_overlaps(polygons)
    if overlap:
        overlap_configs.append(n)

print(f'Configs with overlaps: {len(overlap_configs)}')
print(f'Overlap configs: {overlap_configs}')

Checking for overlaps in baseline...


Configs with overlaps: 11
Overlap configs: [16, 20, 40, 61, 69, 79, 99, 102, 112, 161, 166]


In [8]:
# Create hybrid submission: use baseline for valid configs, sample for overlapping ones
print('Creating hybrid submission...')

hybrid_data = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    
    if n in overlap_configs:
        # Use sample submission for this config
        config_df = sample_parsed[sample_parsed['id'].str.startswith(prefix)]
        source = 'sample'
    else:
        # Use baseline for this config
        config_df = baseline_parsed[baseline_parsed['id'].str.startswith(prefix)]
        source = 'baseline'
    
    for _, row in config_df.iterrows():
        hybrid_data.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })

hybrid_df = pd.DataFrame(hybrid_data)
print(f'Hybrid submission shape: {hybrid_df.shape}')

# Calculate hybrid score
hybrid_score, hybrid_sides = calculate_score(hybrid_df)
print(f'Hybrid score: {hybrid_score:.6f}')

Creating hybrid submission...


Hybrid submission shape: (20100, 4)


Hybrid score: 76.162043


In [9]:
# Try to fix overlapping configurations by separating overlapping trees
def fix_overlaps(config_df, max_iterations=100, step=0.01):
    """Try to fix overlaps by pushing overlapping trees apart."""
    result = config_df.copy()
    
    for iteration in range(max_iterations):
        # Create polygons
        polygons = []
        for _, row in result.iterrows():
            poly = create_tree_polygon(row['x'], row['y'], row['deg'])
            polygons.append(poly)
        
        # Find overlapping pairs
        overlap_found = False
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                    overlap_found = True
                    # Get centroids
                    ci = np.array([result.iloc[i]['x'], result.iloc[i]['y']])
                    cj = np.array([result.iloc[j]['x'], result.iloc[j]['y']])
                    
                    # Push apart
                    direction = ci - cj
                    if np.linalg.norm(direction) > 1e-10:
                        direction = direction / np.linalg.norm(direction)
                    else:
                        direction = np.array([1.0, 0.0])
                    
                    result.iloc[i, result.columns.get_loc('x')] += step * direction[0]
                    result.iloc[i, result.columns.get_loc('y')] += step * direction[1]
                    result.iloc[j, result.columns.get_loc('x')] -= step * direction[0]
                    result.iloc[j, result.columns.get_loc('y')] -= step * direction[1]
        
        if not overlap_found:
            break
    
    return result, not overlap_found

# Try to fix overlapping configurations
print('Attempting to fix overlapping configurations...')
fixed_data = []
still_overlapping = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = baseline_parsed[baseline_parsed['id'].str.startswith(prefix)].copy()
    
    if n in overlap_configs:
        # Try to fix
        fixed_config, success = fix_overlaps(config_df, max_iterations=200, step=0.005)
        if not success:
            still_overlapping.append(n)
            # Use sample instead
            config_df = sample_parsed[sample_parsed['id'].str.startswith(prefix)]
        else:
            config_df = fixed_config
    
    for _, row in config_df.iterrows():
        fixed_data.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })

fixed_df = pd.DataFrame(fixed_data)
print(f'Still overlapping after fix attempt: {still_overlapping}')

# Calculate fixed score
fixed_score, fixed_sides = calculate_score(fixed_df)
print(f'Fixed score: {fixed_score:.6f}')

Attempting to fix overlapping configurations...


Still overlapping after fix attempt: [61, 69, 79, 99, 102, 112, 161, 166]


Fixed score: 74.583709


In [10]:
# Verify no overlaps in fixed submission
print('Verifying fixed submission...')
final_overlap_configs = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = fixed_df[fixed_df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlap, pair = check_overlaps(polygons)
    if overlap:
        final_overlap_configs.append(n)

print(f'Configs with overlaps in fixed submission: {len(final_overlap_configs)}')
if final_overlap_configs:
    print(f'Overlap configs: {final_overlap_configs}')

Verifying fixed submission...


Configs with overlaps in fixed submission: 0


In [ ]:
# Apply fix_direction rotation optimization to the fixed submission
def rotate_point(x, y, angle_rad, cx=0, cy=0):
    """Rotate point (x,y) around (cx,cy) by angle_rad radians."""
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    x_new = cos_a * (x - cx) - sin_a * (y - cy) + cx
    y_new = sin_a * (x - cx) + cos_a * (y - cy) + cy
    return x_new, y_new

def get_all_vertices(config_df):
    """Get all vertices of all trees in a configuration."""
    all_vertices = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        coords = np.array(poly.exterior.coords[:-1])
        all_vertices.extend(coords.tolist())
    return np.array(all_vertices)

def get_bbox_side_for_rotation(vertices, angle_rad):
    """Get bounding box side after rotating all vertices by angle_rad."""
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotated = vertices @ np.array([[cos_a, sin_a], [-sin_a, cos_a]])
    
    min_x, max_x = rotated[:, 0].min(), rotated[:, 0].max()
    min_y, max_y = rotated[:, 1].min(), rotated[:, 1].max()
    
    return max(max_x - min_x, max_y - min_y)

def fix_direction(config_df):
    """Find optimal rotation angle to minimize bounding box."""
    vertices = get_all_vertices(config_df)
    
    def objective(angle_deg):
        return get_bbox_side_for_rotation(vertices, np.radians(angle_deg))
    
    result = minimize_scalar(objective, bounds=(0, 90), method='bounded')
    return result.x, result.fun

print('Applying fix_direction optimization...')
optimized_data = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = fixed_df[fixed_df['id'].str.startswith(prefix)].copy()
    
    # Find optimal rotation
    optimal_angle, new_side = fix_direction(config_df)
    
    # Get center of bounding box
    vertices = get_all_vertices(config_df)
    cx = (vertices[:, 0].min() + vertices[:, 0].max()) / 2
    cy = (vertices[:, 1].min() + vertices[:, 1].max()) / 2
    
    angle_rad = np.radians(optimal_angle)
    
    for idx, row in config_df.iterrows():
        new_x, new_y = rotate_point(row['x'], row['y'], angle_rad, cx, cy)
        new_deg = (row['deg'] + optimal_angle) % 360
        
        optimized_data.append({
            'id': row['id'],
            'x': new_x,
            'y': new_y,
            'deg': new_deg
        })
    
    if n % 50 == 0:
        print(f'Config {n} optimized')

optimized_df = pd.DataFrame(optimized_data)
print('Optimization complete')

# Calculate optimized score
optimized_score, optimized_sides = calculate_score(optimized_df)
print(f'Optimized score: {optimized_score:.6f}')

In [7]:
# Check for overlaps in baseline
print('Checking for overlaps in baseline...')
has_overlaps = False
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = baseline_parsed[baseline_parsed['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlap, pair = check_overlaps(polygons)
    if overlap:
        print(f'Config {n} has overlap at pair {pair}')
        has_overlaps = True

if not has_overlaps:
    print('No overlaps found in baseline!')

Checking for overlaps in baseline...
Config 16 has overlap at pair (5, 11)
Config 20 has overlap at pair (0, 16)
Config 40 has overlap at pair (4, 26)


Config 61 has overlap at pair (23, 26)
Config 69 has overlap at pair (44, 59)
Config 79 has overlap at pair (8, 52)


Config 99 has overlap at pair (10, 40)
Config 102 has overlap at pair (15, 20)
Config 112 has overlap at pair (0, 96)


Config 161 has overlap at pair (17, 114)
Config 166 has overlap at pair (44, 123)


In [8]:
# Implement fix_direction rotation optimization
def rotate_point(x, y, angle_rad, cx=0, cy=0):
    """Rotate point (x,y) around (cx,cy) by angle_rad radians."""
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    x_new = cos_a * (x - cx) - sin_a * (y - cy) + cx
    y_new = sin_a * (x - cx) + cos_a * (y - cy) + cy
    return x_new, y_new

def get_all_vertices(config_df):
    """Get all vertices of all trees in a configuration."""
    all_vertices = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        coords = np.array(poly.exterior.coords[:-1])  # Exclude closing point
        all_vertices.extend(coords.tolist())
    return np.array(all_vertices)

def get_bbox_side_for_rotation(vertices, angle_rad):
    """Get bounding box side after rotating all vertices by angle_rad."""
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotated = vertices @ np.array([[cos_a, sin_a], [-sin_a, cos_a]])
    
    min_x, max_x = rotated[:, 0].min(), rotated[:, 0].max()
    min_y, max_y = rotated[:, 1].min(), rotated[:, 1].max()
    
    return max(max_x - min_x, max_y - min_y)

def fix_direction(config_df):
    """Find optimal rotation angle to minimize bounding box."""
    vertices = get_all_vertices(config_df)
    
    # Find optimal angle in [0, 90] degrees
    def objective(angle_deg):
        return get_bbox_side_for_rotation(vertices, np.radians(angle_deg))
    
    result = minimize_scalar(objective, bounds=(0, 90), method='bounded')
    optimal_angle = result.x
    
    return optimal_angle, result.fun

print('fix_direction function defined')

fix_direction function defined


In [9]:
# Apply fix_direction to each configuration and see improvement
print('Applying fix_direction optimization...')

optimized_data = []
improvements = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = baseline_parsed[baseline_parsed['id'].str.startswith(prefix)].copy()
    
    # Get original side
    original_side = baseline_sides[n]
    
    # Find optimal rotation
    optimal_angle, new_side = fix_direction(config_df)
    
    # Apply rotation to all trees
    # Rotate positions around center of bounding box
    vertices = get_all_vertices(config_df)
    cx = (vertices[:, 0].min() + vertices[:, 0].max()) / 2
    cy = (vertices[:, 1].min() + vertices[:, 1].max()) / 2
    
    angle_rad = np.radians(optimal_angle)
    
    for idx, row in config_df.iterrows():
        # Rotate position
        new_x, new_y = rotate_point(row['x'], row['y'], angle_rad, cx, cy)
        # Add rotation to tree angle
        new_deg = (row['deg'] + optimal_angle) % 360
        
        optimized_data.append({
            'id': row['id'],
            'x': new_x,
            'y': new_y,
            'deg': new_deg
        })
    
    improvement = original_side - new_side
    improvements.append(improvement)
    
    if n % 50 == 0:
        print(f'Config {n}: {original_side:.6f} -> {new_side:.6f} (improvement: {improvement:.6f})')

print(f'\nTotal improvements: {sum(improvements):.6f}')
print(f'Configs improved: {sum(1 for i in improvements if i > 0.0001)}')

optimized_df = pd.DataFrame(optimized_data)

Applying fix_direction optimization...


Config 50: 4.247076 -> 4.247077 (improvement: -0.000000)


Config 100: 5.860005 -> 5.860005 (improvement: -0.000001)


Config 150: 7.110523 -> 7.110523 (improvement: -0.000000)


Config 200: 8.216433 -> 8.216434 (improvement: -0.000001)

Total improvements: -0.000048
Configs improved: 0


In [10]:
# Calculate new score
print('Calculating optimized score...')
optimized_score, optimized_sides = calculate_score(optimized_df)
print(f'Optimized score: {optimized_score:.6f}')
print(f'Improvement: {baseline_score - optimized_score:.6f}')

Calculating optimized score...


Optimized score: 70.625924
Improvement: -0.000006


In [11]:
# Check for overlaps in optimized solution
print('Checking for overlaps in optimized solution...')
has_overlaps = False
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = optimized_df[optimized_df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlap, pair = check_overlaps(polygons)
    if overlap:
        print(f'Config {n} has overlap at pair {pair}')
        has_overlaps = True

if not has_overlaps:
    print('No overlaps found in optimized solution!')

Checking for overlaps in optimized solution...
Config 2 has overlap at pair (0, 1)
Config 16 has overlap at pair (5, 11)
Config 18 has overlap at pair (3, 14)
Config 20 has overlap at pair (0, 16)
Config 26 has overlap at pair (14, 24)
Config 31 has overlap at pair (18, 22)
Config 33 has overlap at pair (2, 31)
Config 40 has overlap at pair (4, 26)
Config 42 has overlap at pair (5, 38)
Config 48 has overlap at pair (13, 29)


Config 53 has overlap at pair (13, 48)
Config 61 has overlap at pair (23, 26)
Config 70 has overlap at pair (40, 55)
Config 75 has overlap at pair (4, 8)
Config 79 has overlap at pair (8, 52)


Config 84 has overlap at pair (7, 67)
Config 99 has overlap at pair (3, 91)
Config 102 has overlap at pair (15, 20)
Config 105 has overlap at pair (9, 66)


Config 112 has overlap at pair (0, 96)
Config 127 has overlap at pair (55, 71)


Config 138 has overlap at pair (36, 60)


Config 160 has overlap at pair (26, 88)
Config 161 has overlap at pair (6, 15)
Config 165 has overlap at pair (17, 155)
Config 166 has overlap at pair (91, 150)


Config 184 has overlap at pair (48, 153)


In [12]:
# Implement squeeze operation
def squeeze_config(config_df, scale=0.999):
    """Scale all tree positions toward center."""
    # Find center
    cx = config_df['x'].mean()
    cy = config_df['y'].mean()
    
    result = config_df.copy()
    result['x'] = cx + scale * (config_df['x'] - cx)
    result['y'] = cy + scale * (config_df['y'] - cy)
    
    return result

def validate_config(config_df):
    """Check if configuration has no overlaps."""
    polygons = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlap, _ = check_overlaps(polygons)
    return not overlap

def squeeze_until_overlap(config_df, min_scale=0.98, step=0.0005):
    """Squeeze configuration until overlap, then back off."""
    scale = 1.0
    best_valid = config_df.copy()
    
    while scale > min_scale:
        squeezed = squeeze_config(config_df, scale)
        if validate_config(squeezed):
            best_valid = squeezed.copy()
            scale -= step
        else:
            break
    
    return best_valid

print('Squeeze functions defined')

Squeeze functions defined


In [ ]:
# Apply squeeze to each configuration
print('Applying squeeze optimization...')

squeezed_data = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = optimized_df[optimized_df['id'].str.startswith(prefix)].copy()
    
    # Apply squeeze
    squeezed = squeeze_until_overlap(config_df, min_scale=0.98, step=0.001)
    
    for _, row in squeezed.iterrows():
        squeezed_data.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })
    
    if n % 50 == 0:
        print(f'Config {n} squeezed')

squeezed_df = pd.DataFrame(squeezed_data)
print('Squeeze complete')

In [ ]:
# Calculate squeezed score
print('Calculating squeezed score...')
squeezed_score, squeezed_sides = calculate_score(squeezed_df)
print(f'Squeezed score: {squeezed_score:.6f}')
print(f'Improvement from baseline: {baseline_score - squeezed_score:.6f}')

In [ ]:
# Apply fix_direction again after squeeze
print('Applying fix_direction again after squeeze...')

final_data = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = squeezed_df[squeezed_df['id'].str.startswith(prefix)].copy()
    
    # Find optimal rotation
    optimal_angle, new_side = fix_direction(config_df)
    
    # Apply rotation
    vertices = get_all_vertices(config_df)
    cx = (vertices[:, 0].min() + vertices[:, 0].max()) / 2
    cy = (vertices[:, 1].min() + vertices[:, 1].max()) / 2
    
    angle_rad = np.radians(optimal_angle)
    
    for idx, row in config_df.iterrows():
        new_x, new_y = rotate_point(row['x'], row['y'], angle_rad, cx, cy)
        new_deg = (row['deg'] + optimal_angle) % 360
        
        final_data.append({
            'id': row['id'],
            'x': new_x,
            'y': new_y,
            'deg': new_deg
        })
    
    if n % 50 == 0:
        print(f'Config {n} rotated')

final_df = pd.DataFrame(final_data)
print('Final rotation complete')

In [ ]:
# Calculate final score
print('Calculating final score...')
final_score, final_sides = calculate_score(final_df)
print(f'Final score: {final_score:.6f}')
print(f'Improvement from baseline: {baseline_score - final_score:.6f}')

In [ ]:
# Final overlap check
print('Final overlap check...')
has_overlaps = False
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = final_df[final_df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        poly = create_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlap, pair = check_overlaps(polygons)
    if overlap:
        print(f'Config {n} has overlap at pair {pair}')
        has_overlaps = True

if not has_overlaps:
    print('No overlaps found in final solution!')

In [ ]:
# Create submission file
submission = final_df.copy()

# Format with 's' prefix and high precision
for col in ['x', 'y', 'deg']:
    submission[col] = 's' + submission[col].apply(lambda x: f'{x:.15f}')

submission.to_csv('/home/code/experiments/001_baseline/submission.csv', index=False)
submission.to_csv('/home/submission/submission.csv', index=False)

print('Submission saved!')
print(submission.head())

In [ ]:
# Summary
print('\n' + '='*50)
print('SUMMARY')
print('='*50)
print(f'Baseline score: {baseline_score:.6f}')
print(f'After fix_direction: {optimized_score:.6f}')
print(f'After squeeze: {squeezed_score:.6f}')
print(f'Final score: {final_score:.6f}')
print(f'Total improvement: {baseline_score - final_score:.6f}')
print('='*50)